## Maximize Sales

In [1]:
! pip install pulp


   ---------------------------------------- 0.0/17.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/17.7 MB 682.7 kB/s eta 0:00:26
   ---------------------------------------- 0.1/17.7 MB 1.3 MB/s eta 0:00:14
   - -------------------------------------- 0.6/17.7 MB 5.8 MB/s eta 0:00:03
   ----- ---------------------------------- 2.6/17.7 MB 16.7 MB/s eta 0:00:01
   ----------- ---------------------------- 4.9/17.7 MB 22.3 MB/s eta 0:00:01
   ---------------- ----------------------- 7.1/17.7 MB 26.9 MB/s eta 0:00:01
   -------------------- ------------------- 9.2/17.7 MB 29.6 MB/s eta 0:00:01
   ------------------------ --------------- 11.0/17.7 MB 43.7 MB/s eta 0:00:01
   ----------------------------- ---------- 12.9/17.7 MB 46.9 MB/s eta 0:00:01
   ---------------------------------- ----- 15.1/17.7 MB 43.5 MB/s eta 0:00:01
   ---------------------------------------  17.4/17.7 MB 46.7 MB/s eta 0:00:01
   ---------------------------------------- 17.7/17.7 MB 40.9 MB/s et

In [20]:
##########################
# I. INSTALL & IMPORT
##########################
# pip install pulp pandas numpy
import pandas as pd
import numpy as np
from pulp import LpProblem, LpMaximize, LpVariable, LpStatus, value

##########################
# II. LOAD DATA
##########################
# Your CSV must have columns:
# [
#   'Brand', 'Segment', 'Initial Sales', 'Margin %',
#   'Min Trend %', 'Max Trend %',
#   'Min Contribution %', 'Max Contribution %'
#   ... plus any others like Portfolio, Geography, Category, etc.
# ]
data_file = "Acme_Synthetic_Dataset.csv"
df = pd.read_csv(data_file)

print("Head of dataset:")
print(df.head())
print("\nColumns:", df.columns.tolist())

##########################
# III. BUILD THE MODEL
##########################
# Create a linear problem instance to maximize total sales
model = LpProblem("Maximize_Total_Sales", LpMaximize)

# We have two sets of decision variables:
# (1) x_vars for "New Sales" of each segment
# (2) X_vars for "Total sales" of each brand

# Ensure each row in df has a unique ID
df.reset_index(drop=False, inplace=True)
df.rename(columns={"index": "row_id"}, inplace=True)

# Gather the unique brands
brands = df["Brand"].unique()

# Create brand-level total variables
X_vars = {}
for b in brands:
    var_name = f"X_brand_{b}"
    X_vars[b] = LpVariable(var_name, lowBound=0)

# Create segment-level variables
x_vars = {}
for row in df.itertuples():
    var_name = f"x_{row.Brand}_{row.Segment}_{row.row_id}"
    x_vars[row.row_id] = LpVariable(var_name, lowBound=0)

##########################
# IV. ADD CONSTRAINTS
##########################

# A) Trend Constraints:
#    s_i * (1 + minTrend/100) <= x_i <= s_i * (1 + maxTrend/100)
for row in df.itertuples():
    row_id = row.row_id
    seg_var = x_vars[row_id]
    
    # Corrected: Access attributes directly with underscores
    init_sales = row.Initial_Sales
    min_trend = row.Min_Trend
    max_trend = row.Max_Trend
    
    lower_bound = init_sales * (1 + min_trend / 100.0)
    upper_bound = init_sales * (1 + max_trend / 100.0)
    
    model += seg_var >= lower_bound, f"TrendLB_{row_id}"
    model += seg_var <= upper_bound, f"TrendUB_{row_id}"
# B) Brand Summation:
#    sum of x_i for brand b = X_b
for b in brands:
    df_b = df[df["Brand"] == b]
    sum_segments = sum(x_vars[row_b.row_id] for row_b in df_b.itertuples())
    model += (sum_segments == X_vars[b]), f"BrandSum_{b}"

# C) Contribution Constraints:
#    minContrib% * X_b <= x_i <= maxContrib% * X_b
for b in brands:
    df_b = df[df["Brand"] == b]
    for row in df_b.itertuples():
        seg_var = x_vars[row.row_id]
        brand_var = X_vars[b]
        
        min_contrib = row._asdict()["Min_Contribution"]
        max_contrib = row._asdict()["Max_Contribution"]
        
        model += seg_var >= (min_contrib / 100.0) * brand_var, f"ContribMin_{row.row_id}"
        model += seg_var <= (max_contrib / 100.0) * brand_var, f"ContribMax_{row.row_id}"

##########################
# V. OBJECTIVE FUNCTION
##########################
# Maximize the sum of brand totals
model += sum(X_vars[b] for b in brands), "MaximizeTotalSales"

##########################
# VI. SOLVE THE MODEL
##########################
model.solve()

print("\nSolver Status:", LpStatus[model.status])
optimal_total_sales = value(model.objective)
print("Maximum Total Sales Achieved (objective):", optimal_total_sales)

##########################
# VII. EXTRACT RESULTS
##########################
new_sales_list = []
new_trend_list = []
new_contribution_list = []
Margindollars_list = []

for row in df.itertuples():
    row_id = row.row_id
    brand = row.Brand
    
    init_sales = row.Initial_Sales
    # Corrected: Access margin percentage
    Marginpct = row.Margin  # Column 'Margin %' becomes 'Margin'
    
    final_sales = x_vars[row_id].varValue
    brand_total = X_vars[brand].varValue if X_vars[brand].varValue else 1e-9
    
    # Calculate new trend in %
    new_trend = ((final_sales - init_sales) / init_sales) * 100.0
    
    # Calculate new contribution in %
    new_contribution = (final_sales / brand_total) * 100.0
    \
    
    # Calculate margin in $
    Margindollars = final_sales * (Marginpct / 100.0)
    
    new_sales_list.append(final_sales)
    new_trend_list.append(new_trend)
    new_contribution_list.append(new_contribution)
    Margindollars_list.append(Margindollars)

# Append new columns to df
df["New Sales"] = new_sales_list
df["New Trend %"] = new_trend_list
df["New Contribution %"] = new_contribution_list
df["Margin $"] = Margindollars_list

##########################
# VIII. REPORT FINAL RESULTS
##########################
# Sort descending by new sales for clarity

print("\nTop 10 segments by final 'New Sales':")
print(df.head(10))

# Check total new sales from segment level = objective
sum_segments_sales = df["New Sales"].sum()
print("\nCheck total final sales from sum of segments:", sum_segments_sales,
      "\n(Should match the solver's objective above.)")

# Print a subset of columns to show each unit's sales, margin, trend, contribution
desired_columns = [
    "Brand",
    "Segment",
    "Initial_Sales",
    "Margin",
    "New Sales",
    "New Trend %",
    "New Contribution %",
    "Margin $"
]
df_report = df[desired_columns].copy()
print("\nSales, Margin, Trend, and Contribution for each unit:")
print(df_report.head(20))

# Save the final solution to a new CSV
df_report.to_csv("Acme_MaxSales_Solution.csv", index=False)
print("\nSaved results to 'Acme_MaxSales_Solution.csv'.")

sum_segments_sales = df["New Sales"].sum()
print("\nSum of 'New Sales' across all segments:", sum_segments_sales)
print("(Should match the solver's objective above.)")

user_input = float(input("\nEnter a random sample (or target) value for total revenue: "))

if abs(sum_segments_sales - user_input) < 1e-6:
    print("Yes, your input matches the total revenue exactly.")
else:
    difference = sum_segments_sales - user_input
    if difference > 0:
        print(f"No, the total revenue ({sum_segments_sales:.2f}) is HIGHER than your input by {difference:.2f}.")
    else:
        print(f"No, the total revenue ({sum_segments_sales:.2f}) is LOWER than your input by {-difference:.2f}.")

df.sort_values(by="New Sales", ascending=False, inplace=True)


Head of dataset:
   Portfolio      Geography         Category           Brand    Segment  \
0  Hair/APDO           Asia            Tools         Balmain  Fragrance   
1  Hair/APDO  South America     Face Make-Up         Balmain  Fragrance   
2  Hair/APDO         Europe            Tools  Frederic Malle    Bronzer   
3  Skin/Body  North America  Make Up Brushes  Frederic Malle  Fragrance   
4  Hair/APDO  South America         Hair Dye     Bobbi Brown    Bronzer   

   Initial_Sales  Margin  Min_Trend  Max_Trend  Min_Contribution  \
0        2734489      50          0         15                 4   
1        1739911      33          0         15                 4   
2        3844769      59         -3          4                 4   
3        1417040      41         -3          4                 5   
4        2000942      45         -1          3                 7   

   Max_Contribution  
0                30  
1                30  
2                30  
3                40  
4            